In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

In [12]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [14]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [ ]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [16]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [18]:
#  Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
# Save the enocders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression Problem Statement

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
# Build Our ANN model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with input layer
    Dense(32, activation='relu'), # HL2
    Dense(1) # Output layer for regression default activation is linear
]
)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
# Set up the Tensorboard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [25]:
# Train the model
history = model.fit(
    X_train, y_train, validation_data =(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 7ms/step - loss: 100384.9531 - mae: 100384.9531 - val_loss: 98542.8047 - val_mae: 98542.8047
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99712.2969 - mae: 99712.2969 - val_loss: 97178.9844 - val_mae: 97178.9844
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 97311.0234 - mae: 97311.0234 - val_loss: 93659.5859 - val_mae: 93659.5859
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 92574.0938 - mae: 92574.0938 - val_loss: 87766.1250 - val_mae: 87766.1250
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 85587.3047 - mae: 85587.3047 - val_loss: 80043.1484 - val_mae: 80043.1484
Epoch 6/100
250/250 [==============================] - 2s 7ms/step - loss: 77225.5781 - mae: 77225.5781 - val_loss: 71595.3906 - val_mae: 71595.3906
Epoch 7/100
250/250 [==============================] - 3s 11ms/step - loss: 68728.9531 - mae: 68728.95

In [26]:
# Load tensorboard extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 21972), started 0:00:14 ago. (Use '!kill 21972' to kill it.)

In [30]:
# Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

63/63 [==============================] - 0s 2ms/step - loss: 50343.8242 - mae: 50343.8242
Test Loss: 50343.82421875, Test MAE: 50343.82421875


In [31]:
model.save('regression_model.h5')

f:\MachineLearning\ANN CLASSIFICATION\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
